In [ ]:
import os
import sys
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "3"
import matplotlib.pyplot as plt

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
from torch.autograd import Variable
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import random
import time

In [ ]:
pretrained = "twitter.27B.50d"

In [ ]:
name = ".".join(pretrained.split('.')[:2])
dim = int(pretrained.split('.')[-1].replace('d',''))

In [ ]:
sys.path.append('..')
from datasets import imdb as dataset
import TextCNN.models.textcnn as textcnn

In [ ]:
from vae import *

In [ ]:
dataset_name = 'rt'

In [ ]:
if dataset_name == 'imdb':
    dataset.split_train_valid(path_data='/ksozykinraid/data/nlp/IMDB/imdb.csv')
    pth = torch.load('/ksozykinraid/data/nlp/IMDB/imdb_text_cnn_best_twitter27b50d.pth',map_location='cpu')
elif dataset_name == 'rt':
    dataset.split_train_valid(path_data='/ksozykinraid/data/nlp/rt-polaritydata/rt-polarity.csv')
    pth = torch.load('/ksozykinraid/data/nlp/rt-polaritydata/rt-polarity_twitter.27B.50d_text_cnn_best.pth',map_location='cpu')

In [ ]:
fix_length=32
batch_size = 64

In [ ]:
dataset = dataset.IMDB_Dataset(pretrained=pretrained,fix_length=fix_length,mbsize=batch_size)

In [ ]:
V = dataset.TEXT.vocab

In [ ]:
target = textcnn.textCNN(V, 100, [3,4,5], 0.5 , 2)

In [ ]:
target

In [ ]:
target.load_state_dict(pth['state_dict'])

In [ ]:
target.embed = feedEmbed(target.embed )

In [ ]:
target

In [ ]:
device='cuda'
disc = textcnn.textCNN(V, 100, [3,4,5], 0.5 , 2)
disc.embed = target.embed 
disc = disc.to(device)

In [ ]:
disable_all_grads(target)

In [ ]:
device='cuda'

In [ ]:
target = target.to(device)

In [ ]:
#model.py


In [ ]:
if not os.path.exists('pth/'):
    os.makedirs('pth/')


In [ ]:
itos = dataset.LABEL.vocab.itos

In [ ]:
h_dim = 32
lr = 1e-2
lr_decay_every = 1000

In [ ]:
dataset.n_train_batch

In [ ]:


model = RNN_VAE(
    dataset.n_vocab, h_dim,h_dim,2, p_word_dropout=0.3,
    pretrained_embeddings=dataset.get_vocab_vectors(), freeze_embeddings=False,max_sent_len=fix_length,
    gpu=True
)


In [ ]:
def batch_itos(x):
    return [" ".join([V.itos[e] for e in s]) for s in x]

In [ ]:
def perplexity(p):
    # [seq_len,probs]
    # per 1 elem in batch
    # as in paper
    with torch.no_grad():
        #ppl = -torch.log(F.softmax(p,-1)).sum(1).sum(0)/len(V)
        ppl = -F.log_softmax(p,-1).sum(1).sum(0)/len(V)
        return ppl.cpu().numpy()

In [ ]:
def save_model(model,phi):
    pth = {}
    pth['state_dict'] = model.state_dict()
    torch.save(pth, 'pth/vae_phi{}.pth'.format(phi))

In [ ]:
def test_attack_rate(N):
    j = 0
    rate = []
    perp = []
    for _ in range(N):
        z = model.sample_z_prior(1)
        c = model.sample_c_prior(1)
        z=torch.cat([z,model.C[torch.argmax(c)].unsqueeze(0)],-1)


        sample_idxs,sample_logits = model.sample_sentence(z, c)
        sample_sent = dataset.idxs2sentence(sample_idxs)
        ppl = perplexity(sample_logits)
        sample_logits = sample_logits.unsqueeze(0)


        if sample_logits.shape[1] < 5:
            continue
        j += 1
        if j >= N:
            break
        c_idx = torch.argmax(c,-1).cpu().numpy()[0]
        pred_label_idx =  torch.argmax(target(sample_logits),-1)[0].cpu().numpy()
        rate.append(c_idx !=  pred_label_idx)
        perp.append(ppl)
    perp = np.mean(ppl)
    attack_rate=np.mean(rate)
    return attack_rate,perp

In [ ]:
with_disriminator = True

In [ ]:

def train(phi = 0,
          num_epochs=5,
          log_interval = 10,
          gen_batch_size=8):
    vanila = (phi == 0)
    print("phi={}".format(phi))
    n_iter = num_epochs*dataset.n_train_batch
    # Annealing for KL term
    kld_start_inc = 1000
    kld_weight = 0.0
    kld_max = 1
    kld_inc = (kld_max - kld_weight) / (n_iter - kld_start_inc)
    it = 0
    loss_data = []
    recon_loss_data = []
    kl_loss_data = []
    opt = optim.Adam(model.vae_params + list(target.parameters()), lr=lr)
    opt_disc = optim.Adam(disc.parameters(), lr=lr)
    for epoch in range(num_epochs):
      #print("epoch",epoch)
      for idx,batch in enumerate(dataset.train_iter):
            x, labels = batch.text, batch.label
            c  = torch.eye(2)[labels]
            x, labels = x.to(device), labels.to(device)
            c = c.to(device)

            recon_loss, kl_loss, x_hat = model.forward(x,c)
            loss = recon_loss + kld_weight * kl_loss


            #Anneal kl_weight
            if it > kld_start_inc and kld_weight < kld_max:
                kld_weight += kld_inc

            loss = loss.reshape(1)
            attack_rate = 0

            if not vanila:
                x_adv, labels_adv = model.generate_sentences(gen_batch_size)
                pred_adv = target(x_adv)
                Ladv = F.cross_entropy(pred_adv,torch.argmax(labels_adv.long(),-1))

                labels_adv_idx =  torch.argmax(labels_adv.long(),-1).cpu().numpy()
                pred_label_idx =  torch.argmax(pred_adv,-1).cpu().numpy()

                attack_rate=np.mean((labels_adv_idx !=  pred_label_idx))
                
                loss -= phi*Ladv
                
                if with_disriminator:
                    x_real = torch.eye(len(V))[x].to(device)
                    x_real = torch.transpose(x_real,0, 1)
                    pred_real = disc(x_real)
                    Ldisc_real = F.cross_entropy(pred_real,labels)
                    Ldisc_real.backward()
                    #https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
                    pred_fake = disc(x_adv.detach())
                    Ldisc_fake = F.cross_entropy(pred_fake,torch.argmax(labels_adv.long(),-1))
                    Ldisc_fake.backward()
                    
                    Ldisc = Ldisc_real + Ldisc_fake
                    grad_norm = torch.nn.utils.clip_grad_norm(disc.parameters(), 1)
                    opt_disc.step()
                    opt_disc.zero_grad()
                    
                
                #print('loss',loss.data)

            loss.backward()
            #grad_norm = torch.nn.utils.clip_grad_norm(model.vae_params, 1)
            opt.step()
            opt.zero_grad()

            loss_data.append(loss.data)
            recon_loss_data.append(recon_loss.reshape(1).data)
            kl_loss_data.append(kl_loss.reshape(1).data)


            if it % log_interval == 0:

                save_model(model,phi)
                x_text = batch_itos(x)
                x_text_hat = batch_itos(torch.argmax(x_hat,-1))

                z = model.sample_z_prior(1)
                c = model.sample_c_prior(1)
                z=torch.cat([z,model.C[torch.argmax(c)].unsqueeze(0)],-1)

                sample_idxs,sample_logits = model.sample_sentence(z, c)
                sample_sent = dataset.idxs2sentence(sample_idxs)

                sample_logits = sample_logits.unsqueeze(0)


                if sample_logits.shape[1] < 5:
                    continue



                pred_label_idx =  torch.argmax(target(sample_logits),-1)[0]


                c_idx = torch.argmax(c)
                print('epoch-%d Iter-%d; Loss: %9.4f; Recon: %9.4f; KL: %9.4f;'%(epoch,it, loss.data, 
                                                                                 recon_loss.data, kl_loss.data))
                print(sample_sent)
                print('sampled',itos[c_idx],'predicted',itos[pred_label_idx])
                print('attack_rate per batch',attack_rate)
                print('perplexity per sampled sentance',perplexity(sample_logits[0]))
                print()
            # Anneal learning rate
            #new_lr = lr * (0.5 ** (it // lr_decay_every))
            #for param_group in opt.param_groups:
            #    param_group['lr'] = new_lr
            it += 1
    
    return loss_data,recon_loss_data, kl_loss_data 

In [ ]:
log_interval =  1000

In [ ]:
loss_data,recon_loss_data, kl_loss_data = train(phi=0,num_epochs=250,log_interval = log_interval)
a,ppl = test_attack_rate(500)
print(a)


In [ ]:
LD = []
RLD = []
KLD = []
for i in range(0, len(loss_data)):
    LD.append(loss_data[i].data.cpu().numpy()[0])
    RLD.append(recon_loss_data[i].data.cpu().numpy()[0])
    KLD.append(kl_loss_data[i].data.cpu().numpy()[0])

In [ ]:

plt.figure(facecolor='white')
plt.plot(LD, marker='o',label='LD')
plt.plot(RLD, marker='o',label='RLD')
plt.plot(KLD, marker='o',label='KLD')
plt.grid()
plt.legend()

In [ ]:

phis=[1,3,6,9]
for phi in phis:
    vae_phi0_pth = torch.load('./pth/vae_phi0.pth',map_location='cpu')
    model.load_state_dict(vae_phi0_pth['state_dict'])
    log_interval = 10
    num_epochs = 3
    loss_data,recon_loss_data, kl_loss_data = train(phi=phi,num_epochs=num_epochs,log_interval = log_interval)
    a,ppl = test_attack_rate(100)
    print('test attack rate',a)
    print('test ppl',ppl)
    #attack_rates_test.append(a)

In [ ]:
phis=[0,1,3,6,9]
at = []
perp = []
for phi in phis:
    print(phi)
    vae_phi0_pth = torch.load('./pth/vae_phi{}.pth'.format(phi),map_location='cpu')
    model.load_state_dict(vae_phi0_pth['state_dict'])
    a,ppl = test_attack_rate(1000)
    print('test attack rate',a)
    print('test ppl',ppl)
    perp.append(ppl)
    at.append(a)

In [ ]:

plt.figure(facecolor='white')
plt.plot(phis,at,marker='o',color='red')
plt.grid()
plt.xlabel("phi")
plt.ylabel("attack rate")

In [ ]:
plt.figure(facecolor='white')
plt.plot(phis,perp,marker='o',color='g')
plt.grid()
plt.xlabel("phi")
plt.ylabel("ppl")